<a href="https://colab.research.google.com/github/MunySafitri/Retrieval_Information/blob/main/SVM_CLASSIFIER_PENYAKIT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [57]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score

In [58]:
np.random.seed(500)

In [59]:
Corpus = pd.read_excel(r'/content/dataset_final-sample.xlsx')

In [60]:
  import nltk
  nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [61]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [62]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [63]:
 nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [64]:
for entry in Corpus['tanda_dan_gejala']: 
  print(entry)
  print(type(entry))

Hidung keluar lendir, choncae nasal kebiruan, bersin-bersin
<class 'str'>
Hidung berlendir, demam, choncae nasal kemerahan, bersin-bersin
<class 'str'>
Hidung bau, nyeri di dalam hidung, ada krusta di dalam hidung
<class 'str'>
Gatal pada hidung, mata, tenggorokan, kulit, atau area apa pun. Batuk. Bersin. Hidung meler. Hidung tersumbat. Tenggorokan gatal. Mata berair.
<class 'str'>
Bersin. Hidung meler. Gatal pada hidung, mata, tenggorokan, kulit, atau area apa pun. Batuk. Hidung tersumbat. Tenggorokan gatal. Mata berair. Lingkaran hitam di bawah mata. Sakit kepala terus-menerus. Gejala sejenis eksim, misalnya kulit sangat kering dan gatal serta sering melepuh. Gatal-gatal. Kelelahan yang amat sangat, 
<class 'str'>
Pilek. Hidung tersumbat. Gatal-gatal pada hidung. Bersin-bersin. Batuk. Mata gatal dan berair. Sakit kepala, 
<class 'str'>
Hidung keluar lendir, choncae nasal kebiruan, bersin-bersin, gatal pada hidung
<class 'str'>
Nyeri perut sebelah kanan bawah, awalnya nyeri di sekitar

In [65]:
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [66]:
# Step - a : Remove blank rows if any.
Corpus['tanda_dan_gejala'].dropna(inplace=True)
# Step - b : Change all the tanda_dan_gejala to lower case. This is required as python interprets 'dog' and 'DOG' differently
Corpus['tanda_dan_gejala'] = [entry.lower() for entry in Corpus['tanda_dan_gejala']]
# Step - c : Tokenization : In this each entry in the corpus will be broken into set of words
Corpus['tanda_dan_gejala']= [word_tokenize(entry) for entry in Corpus['tanda_dan_gejala']]
# Step - d : Remove Stop words, Non-Numeric and perfom Word Stemming/Lemmenting.
# WordNetLemmatizer requires Pos tags to understand if the word is noun or verb or adjective etc. By default it is set to Noun
tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV
for index,entry in enumerate(Corpus['tanda_dan_gejala']):
    # Declaring Empty List to store the words that follow the rules for this step
    Final_words = []
    # Initializing WordNetLemmatizer()
    word_Lemmatized = WordNetLemmatizer()
    # pos_tag function below will provide the 'tag' i.e if the word is Noun(N) or Verb(V) or something else.
    for word, tag in pos_tag(entry):
        # Below condition is to check for Stop words and consider only alphabets
        if word not in stopwords.words('english') and word.isalpha():
            word_Final = word_Lemmatized.lemmatize(word,tag_map[tag[0]])
            Final_words.append(word_Final)
    # The final processed set of words for each iteration will be stored in 'tanda_dan_gejala_final'
    Corpus.loc[index,'tanda_dan_gejala_final'] = str(Final_words)

In [67]:
Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(Corpus['tanda_dan_gejala_final'],Corpus['nama_penyakit'],test_size=0.3)

In [68]:
Encoder = LabelEncoder()
Train_Y = Encoder.fit_transform(Train_Y)
Test_Y = Encoder.fit_transform(Test_Y)

In [69]:
Tfidf_vect = TfidfVectorizer(max_features=5000)
Tfidf_vect.fit(Corpus['tanda_dan_gejala_final'])
Train_X_Tfidf = Tfidf_vect.transform(Train_X)
Test_X_Tfidf = Tfidf_vect.transform(Test_X)

In [70]:
print(Tfidf_vect.vocabulary_)

{'hidung': 268, 'keluar': 341, 'lendir': 430, 'choncae': 139, 'nasal': 568, 'kebiruan': 321, 'berlendir': 92, 'demam': 159, 'kemerahan': 347, 'bau': 47, 'nyeri': 583, 'di': 171, 'dalam': 147, 'ada': 1, 'krusta': 402, 'gatal': 241, 'pada': 596, 'mata': 469, 'tenggorokan': 813, 'kulit': 407, 'atau': 33, 'area': 26, 'apa': 25, 'pun': 682, 'batuk': 46, 'bersin': 98, 'meler': 474, 'tersumbat': 833, 'berair': 63, 'lingkaran': 447, 'hitam': 275, 'bawah': 48, 'sakit': 712, 'kepala': 356, 'gejala': 242, 'sejenis': 734, 'eksim': 211, 'misalnya': 547, 'sangat': 719, 'kering': 364, 'dan': 149, 'serta': 765, 'sering': 764, 'melepuh': 473, 'kelelahan': 336, 'yang': 890, 'amat': 15, 'pilek': 661, 'perut': 659, 'sebelah': 726, 'kanan': 309, 'awalnya': 36, 'sekitar': 737, 'daerah': 145, 'pusar': 687, 'lemas': 428, 'dekat': 157, 'bagian': 38, 'atas': 32, 'menjadi': 519, 'semakin': 751, 'tajam': 795, 'ketika': 377, 'bergerak': 76, 'ke': 316, 'ini': 285, 'penting': 633, 'tanda': 801, 'utama': 874, 'biasan

In [71]:
print(Train_X_Tfidf)

  (0, 568)	0.4752177371886978
  (0, 347)	0.34195960291103045
  (0, 268)	0.32357812142857517
  (0, 159)	0.17193850566845262
  (0, 139)	0.4752177371886978
  (0, 92)	0.5451001596960795
  (1, 709)	0.2677087627577838
  (1, 562)	0.2789538477521666
  (1, 559)	0.2789538477521666
  (1, 548)	0.2789538477521666
  (1, 535)	0.2485501546120657
  (1, 456)	0.24570000640454542
  (1, 364)	0.32382644531329274
  (1, 270)	0.4500182748886876
  (1, 173)	0.3118767602602772
  (1, 159)	0.16282108947699464
  (1, 149)	0.15066362877154976
  (1, 107)	0.25451915281470416
  (1, 46)	0.23260102093396534
  (2, 833)	0.2072419461672148
  (2, 813)	0.31892402782105045
  (2, 682)	0.2729504478819203
  (2, 596)	0.10766461364397534
  (2, 474)	0.24507142415767982
  (2, 469)	0.29826773357077563
  :	:
  (113, 610)	0.1924667901851696
  (113, 573)	0.1924667901851696
  (113, 572)	0.1924667901851696
  (113, 545)	0.14311789416628498
  (113, 517)	0.1924667901851696
  (113, 397)	0.1924667901851696
  (113, 379)	0.15335871536420975
  (113,

In [72]:
# fit the training dataset on the NB classifier
Naive = naive_bayes.MultinomialNB()
Naive.fit(Train_X_Tfidf,Train_Y)
# predict the labels on validation dataset
predictions_NB = Naive.predict(Test_X_Tfidf)
# Use accuracy_score function to get the accuracy
print("Naive Bayes Accuracy Score -> ",accuracy_score(predictions_NB, Test_Y)*100)

Naive Bayes Accuracy Score ->  8.16326530612245


In [73]:
# Classifier - Algorithm - SVM
# fit the training dataset on the classifier
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(Train_X_Tfidf,Train_Y)
# predict the labels on validation dataset
predictions_SVM = SVM.predict(Test_X_Tfidf)
# Use accuracy_score function to get the accuracy
print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, Test_Y)*100)

SVM Accuracy Score ->  10.204081632653061
